## 目標：透過selenium套件來抓取FB社團中的圖片圖檔並下載

### 已知問題：
1. FB的圖片共分成縮圖與原圖的連結，2者並不相同。

In [1]:
import requests
import warnings #用於取消音軟體版本而出現的警告
warnings.filterwarnings('ignore')
import time

# bs4來處理抓取下來的網頁資料
from bs4 import BeautifulSoup

# 透過selenium來操作網頁
from selenium import webdriver

# 透過selenium來操作、輸入、滑動網頁
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# 輸入FB帳號密碼
import getpass

username = input("輸入FaceBook個人帳號：")

userpass = getpass.getpass("輸入個人密碼：")

輸入FaceBook個人帳號：000000@gmail.com
輸入個人密碼：


In [3]:
# chrome_options.add_argument() 添加參數
options = webdriver.ChromeOptions()

# options.add_argument("--disable-infobars") 此方法已失效，要改用下列做法，主要去除掉由自動操作Chrome所跳出的提示。
options.add_experimental_option("excludeSwitches", ['enable-automation']);

# 視窗最大化
options.add_argument("--start-maximized")

# 規避bug，文檔提到
options.add_argument('--disable-gpu') 

prefs = {
    # 禁用瀏覽器彈窗
    "profile.default_content_setting_values":{
    "notifications" : 2  },
    # 禁用彈出儲存密碼視窗
    "profile.password_manager_enabled": False, "credentials_enable_service": False
}
options.add_experimental_option('prefs',prefs)

# options.add_argument()

In [16]:
driver = webdriver.Chrome(options = options)
driver.get("https://www.facebook.com/")

driver.find_element("id", "email").send_keys(username)
driver.find_element("id", "pass").send_keys(userpass)
driver.find_element("name", "login").submit()

# 登入後等待幾秒
login_wating = 3
# 宣告後在driver整個生命週期都生效
driver.implicitly_wait(login_wating)

In [ ]:
driver.close()

In [65]:
# 接下來指定特定社團的頁面
fb_club = "https://www.facebook.com/groups/182779590286685"

driver.get(fb_club)

#driver.implicitly_wait(login_wating)

In [66]:
# 開始抓取網頁中指定Tag中圖片的連結

max_scrollpages = 3 # 欲爬取的頁面數目，此頁面數字可以調整。

sec_waiting = 1    # 暫停駐留在一個頁面的時間，以秒為單位。


for repeated_action in range(1, max_scrollpages + 1):
    soup = BeautifulSoup(driver.page_source, 'lxml') # 使用 BeautifulSoup() 來解析取回頁面，解釋模式選擇使用 'lxml' 模式。
    
    photos_link = soup.find_all("a",{"class":"x1i10hfl x1qjc9v5 xjbqb8w xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 "\
                                     "x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xdl72j9 x2lah0s "\
                                     "xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli xexx8yu x4uap5 x18d9i69 "\
                                     "xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x1o1ewxj x3x9cwd x1e5q0jg "\
                                     "x13rtm0m x1q0g3np x87ps6o x1lku1pv x1rg5ohu x1a2a7pz x1ey2m1c xds687c x10l6tqk "\
                                     "x17qophe x13vifvy x1pdlv7q"})
    
    time.sleep(sec_waiting)
    
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    
print("共抓取到 {} 個連結".format(len(photos_link)))

共抓取到 15 個連結


In [67]:
import pandas as pd

df = pd.DataFrame(columns = ["圖片點擊連結"])

links_ls = []
for links in photos_link:
    links_ls.append(links['href'])


df = pd.DataFrame(links_ls, columns = ["圖片點擊連結"])

df.index += 1

In [68]:
print(type(df))

print(type(links))

print(type(photos_link))

print(type(links_ls))

<class 'pandas.core.frame.DataFrame'>
<class 'bs4.element.Tag'>
<class 'bs4.element.ResultSet'>
<class 'list'>


In [69]:
df.style\
    .set_table_styles([ dict(selector = 'th', props = [('text-align', 'center')])])\
    .set_properties(**{'text-align': 'left', 'font-weight': 'bold'})

,圖片點擊連結
1,https://www.facebook.com/photo/?fbid=3000978010046973&set=pcb.746491193915519&__cft__[0]=AZV5yrtLyFCO-k_0wHBKpXRQalNgDnokoqmkU_2Rr9a8EEgwS5gyB0qLsUuVUrmn-HPKUoKsBBnvWOjhL4m9a6zuLQpEikr4n5-aM-CWKuatlYhrn4j-rr-_4SAqfoWg0HRtWOh0_Wu63kNtuxugAzWnOUrLRyz1CmqPWTFauT87HAKDKrY0hv5Rg_I5SYycI2Q&__tn__=*bH-R
2,https://www.facebook.com/photo/?fbid=3000978070046967&set=pcb.746491193915519&__cft__[0]=AZV5yrtLyFCO-k_0wHBKpXRQalNgDnokoqmkU_2Rr9a8EEgwS5gyB0qLsUuVUrmn-HPKUoKsBBnvWOjhL4m9a6zuLQpEikr4n5-aM-CWKuatlYhrn4j-rr-_4SAqfoWg0HRtWOh0_Wu63kNtuxugAzWnOUrLRyz1CmqPWTFauT87HAKDKrY0hv5Rg_I5SYycI2Q&__tn__=*bH-R
3,https://www.facebook.com/photo/?fbid=3000977973380310&set=pcb.746491193915519&__cft__[0]=AZV5yrtLyFCO-k_0wHBKpXRQalNgDnokoqmkU_2Rr9a8EEgwS5gyB0qLsUuVUrmn-HPKUoKsBBnvWOjhL4m9a6zuLQpEikr4n5-aM-CWKuatlYhrn4j-rr-_4SAqfoWg0HRtWOh0_Wu63kNtuxugAzWnOUrLRyz1CmqPWTFauT87HAKDKrY0hv5Rg_I5SYycI2Q&__tn__=*bH-R
4,https://www.facebook.com/photo/?fbid=3000978013380306&set=pcb.746491193915519&__cft__[0]=AZV5yrtLyFCO-k_0wHBKpXRQalNgDnokoqmkU_2Rr9a8EEgwS5gyB0qLsUuVUrmn-HPKUoKsBBnvWOjhL4m9a6zuLQpEikr4n5-aM-CWKuatlYhrn4j-rr-_4SAqfoWg0HRtWOh0_Wu63kNtuxugAzWnOUrLRyz1CmqPWTFauT87HAKDKrY0hv5Rg_I5SYycI2Q&__tn__=*bH-R
5,https://www.facebook.com/photo/?fbid=3000978136713627&set=pcb.746491193915519&__cft__[0]=AZV5yrtLyFCO-k_0wHBKpXRQalNgDnokoqmkU_2Rr9a8EEgwS5gyB0qLsUuVUrmn-HPKUoKsBBnvWOjhL4m9a6zuLQpEikr4n5-aM-CWKuatlYhrn4j-rr-_4SAqfoWg0HRtWOh0_Wu63kNtuxugAzWnOUrLRyz1CmqPWTFauT87HAKDKrY0hv5Rg_I5SYycI2Q&__tn__=*bH-R
6,https://www.facebook.com/photo/?fbid=3000201433457964&set=pcb.745939793970659&__cft__[0]=AZVQrPbNmffMcVeymW4orgrj9jxjCgH0zI5zcXqY1Zn_gHovqrwwiNg0QPhwMIPrfWZDSYyMESpT7c4Kl0tXmj3iRJ-haZiASmRU8UBl60GCbXtc-TZPjX9aE6_zHXYSn7Te6L4TnVUgKeRpeuq-AWC8kvpgJqiVScvVF9AmfhjNVUK90AwGwneFC-BVLpBQIh0&__tn__=*bH-R
7,https://www.facebook.com/photo/?fbid=3000201300124644&set=pcb.745939793970659&__cft__[0]=AZVQrPbNmffMcVeymW4orgrj9jxjCgH0zI5zcXqY1Zn_gHovqrwwiNg0QPhwMIPrfWZDSYyMESpT7c4Kl0tXmj3iRJ-haZiASmRU8UBl60GCbXtc-TZPjX9aE6_zHXYSn7Te6L4TnVUgKeRpeuq-AWC8kvpgJqiVScvVF9AmfhjNVUK90AwGwneFC-BVLpBQIh0&__tn__=*bH-R
8,https://www.facebook.com/photo/?fbid=3000201426791298&set=pcb.745939793970659&__cft__[0]=AZVQrPbNmffMcVeymW4orgrj9jxjCgH0zI5zcXqY1Zn_gHovqrwwiNg0QPhwMIPrfWZDSYyMESpT7c4Kl0tXmj3iRJ-haZiASmRU8UBl60GCbXtc-TZPjX9aE6_zHXYSn7Te6L4TnVUgKeRpeuq-AWC8kvpgJqiVScvVF9AmfhjNVUK90AwGwneFC-BVLpBQIh0&__tn__=*bH-R
9,https://www.facebook.com/photo/?fbid=3000201696791271&set=pcb.745939793970659&__cft__[0]=AZVQrPbNmffMcVeymW4orgrj9jxjCgH0zI5zcXqY1Zn_gHovqrwwiNg0QPhwMIPrfWZDSYyMESpT7c4Kl0tXmj3iRJ-haZiASmRU8UBl60GCbXtc-TZPjX9aE6_zHXYSn7Te6L4TnVUgKeRpeuq-AWC8kvpgJqiVScvVF9AmfhjNVUK90AwGwneFC-BVLpBQIh0&__tn__=*bH-R
10,https://www.facebook.com/photo/?fbid=3000201496791291&set=pcb.745939793970659&__cft__[0]=AZVQrPbNmffMcVeymW4orgrj9jxjCgH0zI5zcXqY1Zn_gHovqrwwiNg0QPhwMIPrfWZDSYyMESpT7c4Kl0tXmj3iRJ-haZiASmRU8UBl60GCbXtc-TZPjX9aE6_zHXYSn7Te6L4TnVUgKeRpeuq-AWC8kvpgJqiVScvVF9AmfhjNVUK90AwGwneFC-BVLpBQIh0&__tn__=*bH-R


In [70]:
print(len(links_ls))

15


In [71]:
import random
dirver_delay = [2, 2.5, 3.5, 5]
requests_delay = [1, 3, 5, 7]

imgs_ls = []

print("抓取圖片中...")
for action_href_link in range(len(links_ls)):
    
    # 使用links_ls的串列
    driver.get(links_ls[action_href_link])
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    imgs_link = soup.find_all("img", {"class":"x85a59c x193iq5w x4fas0m x19kjcj4"})
    
    for imgs in imgs_link:
        imgs_ls.append(imgs['src'])
    
    time_delay = dirver_delay[random.randint(0, 3)]
    time.sleep(time_delay)

print("抓取完畢")
imgs_ls
    
#try:

    #requests.get(df[action_href_link],timeout = 2)

    #time_delay = delay[random.randint(0, 3)]
    #time.sleep(time_delay)

# except:requests.ConnectionError, err:
        #print("Response："err)

#print("delay{}秒".format(time_delay))

抓取圖片中...
抓取完畢


['https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/359788075_3000977986713642_5570418603541203949_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=sPKq1FdIkhwAX-NjyJc&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfA0sEoDNWgLPiaCbcjuSL7nzXsfwf7_qLptgGSMWHpt9w&oe=64B1BCE9',
 'https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358408013_3000978053380302_4828338804286665469_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=bgoQITz5vf4AX90uOzE&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfDOmDM3Y-LkBiEa8oZPNSfHNeQ_37cby-YGHx7Rx8Yq7A&oe=64B0E734',
 'https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358569843_3000977966713644_3838970978353992721_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=Vp1cWFlDblEAX-9iEFd&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfAb_BdklyFWVXm4HHK9C9nC_lZjBVtUS01BsQvRKpfHxQ&oe=64B1B056',
 'https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358971997_3000977996713641_1303984567827253815_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=7QLz-JwVHEoAX8eUpkg&_nc_ht=scontent.fkhh1-2.fna&oh=

In [72]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
           "Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [73]:
import os
imgs_ls

if not os.path.exists("images"):
        os.mkdir("images")  # 建立資料夾

# for index, link in enumerate(imgs):
for down_links in range(0, len(imgs_ls)):
    
    down_file = requests.get(imgs_ls[down_links]).content
    time.sleep(requests_delay[random.randint(0, 3)])
    try:

        with open("images\\" + "fb_club_FBK{}.jpg".format(down_links + 1), "wb") as fp:
            fp.write(down_file)
            print("下載成功")
            
    except FileNotFoundError:
        
        print("檔案不存在...索引：{}".format(down_links))
    
    except Exception:
        
        print("錯誤下載...索引：{}".format(down_links))


print("下載完畢")

下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載成功
下載完畢


In [75]:
driver.close()

In [11]:
#非重要
url = "https://www.facebook.com/photo/?fbid=2999459080198866&set=pcb.745373874027251&__cft__[0]=AZWzjROWHO5_9jIWTed6pC6tDvOEx4SxaEL1HjmZLRf5QyFyRfYvirwtHBv2WphKhSVOd_FRXy4hrupL5VAhJvjk_NMLRiWUCZct33Q5lSe2Towucq3wOu7pviyXJSzSc01u_E2FwickoUiT6ghyBfwhNuAnSXYcrH0HEoZQQkAIcw60buMlH0klCN81xkGBC7s&__tn__=*bH-R"
driver.get(url)


soup = BeautifulSoup(driver.page_source, 'lxml')

img_con = soup.find_all("img", {"class":"x85a59c x193iq5w x4fas0m x19kjcj4"})

dfs= []
for b in img_con:
    dfs.append(b['src'])

haha = requests.get(dfs[0], headers = headers).content
with open("netflix1.jpg", "wb") as handler: 
    handler.write(haha) 